In [1]:
import wandb

In [2]:
api = wandb.Api()

In [3]:
all_runs = api.runs("betairylia/sim")
no20news = api.sweep("betairylia/sim/e0x2xj0j")
ok20news = api.sweep("betairylia/sim/oshwz2f0")

In [4]:
import json

def IsFinished(run):
    
    if run is None:
        return False
    
    if run.summary is None or run.state is None:
        return False
    
    if 'epoch' not in run.summary:
        return False
    
    return run.summary['epoch'] == 49 and run.state == 'finished'

datasets = ['mnist', 'svhn', 'stl', 'cifar10', 'cifar100', 'omniglot', 'reuters10k', '20news']
dataset_id = {v: i for i, v in enumerate(datasets)}

print(dataset_id)

# Append no20news

def PickFromSweep(all_runs, sweep_id):
    
    result = {}
    
    i = 0
    for run in all_runs:
        if not IsFinished(run):
            print(run.summary)
            print("State: %s" % run.state)
            continue
        if run.sweep is not None and run.sweep.name == sweep_id:
    #         print("=*=*=*=*=*=*=*=*=*=")
    #         print(run.config)
    #         print(run.summary)

    #         print(run.config['dataset'])
    #         print(dataset_id[run.config['dataset']])

    #         print(run.config['mu'])
    #         print(run.config['eta1'])
    #         print(run.config['gamma'])

            obj = {
                'mu': run.config['mu'],
                'gamma': run.config['gamma'],
                'eta1': run.config['eta1'],
                'tau_pos': run.config['tau_pos'],
                'K0': run.config['K0'],
                'dataset': dataset_id[run.config['dataset']]
#                 'accuracy': run.summary['accuracy']
            }
        
            print(json.dumps(obj))
            
            if json.dumps(obj) not in result:
                result[json.dumps(obj)] = 0
            
            result[json.dumps(obj)] += run.summary['accuracy'] / 3
        
            i += 1
    
    print(i)
        
    return result

{'mnist': 0, 'svhn': 1, 'stl': 2, 'cifar10': 3, 'cifar100': 4, 'omniglot': 5, 'reuters10k': 6, '20news': 7}


In [5]:
valid_entries = PickFromSweep(all_runs, 'e0x2xj0j')
valid_entries.update(PickFromSweep(all_runs, 'oshwz2f0'))

{'epoch': 9, 'accuracy': 0.9813428571428572, '_timestamp': 1625105433, 'tau': 0.05, 'loss': -0.4902847111225128, '_step': 132, '_runtime': 196}
State: running
{'epoch': 9, 'tau': 0.05, 'loss': -0.48600146174430847, '_step': 132, '_runtime': 196, 'accuracy': 0.941, '_timestamp': 1625105433}
State: running
{'epoch': 9, 'tau': 0.05, 'loss': -0.4731714725494385, '_step': 131, '_runtime': 195, 'accuracy': 0.9015571428571428, '_timestamp': 1625105432}
State: running


KeyError: 'eta1'

In [ ]:
valid_entries

In [ ]:
def unpack(d):
    result = []
    for k in d:
        obj = json.loads(k)
        obj['accuracy'] = d[k]
        result.append(obj)
    
    return result
        
valid_entries_list = unpack(valid_entries)
valid_entries_list

In [ ]:
max_acc_Di = [0 for i in range(len(datasets))]
for entry in valid_entries_list:
    max_acc_Di[entry['dataset']] = max(max_acc_Di[entry['dataset']], entry['accuracy'])

max_acc_Di

In [ ]:
import numpy as np

results = []

for mu in ['0.1', '0.2']:
    for gamma in ['0.45', '0.55']:
        for eta1 in ['5.0', '6.0']:
            
            max_56_Di = [0 for i in range(len(datasets))]
            
            for entry in valid_entries_list:
                
                if entry['mu'] != mu or entry['gamma'] != gamma or entry['eta1'] != eta1:
                    continue
                
                max_56_Di[entry['dataset']] = max(
                    max_56_Di[entry['dataset']],
                    entry['accuracy'] / max_acc_Di[entry['dataset']]
                )
            
            avg_score = np.asarray(max_56_Di).mean()
            
            results.append('mu %s | gamma %s | eta1 %s = %f' % (mu, gamma, eta1, avg_score))
    
for r in results:
    print(r)